In [3]:
!pip install --no-warn-script-location langchain langchain-community langchain-openai pymysql
# After you run this cell, you need to restart your kernel.
# You only need to run it once per session

In [5]:
from langchain_community.utilities import SQLDatabase

# Connect to the Chinook database
db = SQLDatabase.from_uri("mysql+pymysql://root@localhost:3306/Chinook")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM customer LIMIT 10;")

mysql
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3), (2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5), (3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3), (4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4), (5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4), (6, 'Helena', 'Holý', None, 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', '+420 2 4177 0449', None, 'hholy@gmail.com', 5), (7, 'As

In [6]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)


/var/folders/m2/01jdc6xn2q18pz6crt1_kvph0000gn/T/ipykernel_16625/1086577235.py:11: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(temperature=0),


In [7]:
agent_executor.invoke(
    "What is the best song in our database?",
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track I should query the Track table to find the best song.
Action: sql_db_schema
Action Input: Track
CREATE TABLE `Track` (
	`TrackId` INTEGER NOT NULL, 
	`Name` VARCHAR(200) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci NOT NULL, 
	`AlbumId` INTEGER, 
	`MediaTypeId` INTEGER NOT NULL, 
	`GenreId` INTEGER, 
	`Composer` VARCHAR(220) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci, 
	`Milliseconds` INTEGER NOT NULL, 
	`Bytes` INTEGER, 
	`UnitPrice` DECIMAL(10, 2) NOT NULL, 
	PRIMARY KEY (`TrackId`), 
	CONSTRAINT `FK_TrackAlbumId` FOREIGN KEY(`AlbumId`) REFERENCES `Album` (`AlbumId`), 
	CONSTRAINT `FK_TrackGenreId` FOREIGN KEY(`GenreId`) REFERENCES `Genre` (`GenreId`), 
	CONSTRAINT `FK_TrackMediaTypeId` FOREIGN KEY(`MediaTypeId`) REFERENCES `MediaType` (`MediaTypeId`)
)DEFAULT CHARSET=utf8mb4 ENGINE=In

{'input': 'What is the best song in our database?',
 'output': 'The best song in our database is "Battlestar Galactica: The Story So Far".'}